In [1]:
#%load get_bookinfo.py
#%run get_bookinfo.py
# -*- coding: utf-8 -*-
#________________________________________________
import os
import django
from django.utils import timezone
from django.core.cache import cache
#from django.utils.dateparse import parse_datetime
from datetime import datetime,date#,timezone
import pytz
from bs4 import BeautifulSoup
from bs4.element import NavigableString
from difflib import SequenceMatcher
from IPython.display import clear_output, display
from time import sleep, time
from threading import Thread
from concurrent.futures import ThreadPoolExecutor,ProcessPoolExecutor 

from fake_useragent import UserAgent
from fake_headers import Headers
from pyquery import PyQuery as pq

#
import requests
import pandas as pd
import numpy as np
import random
import re
import json
import csv

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'wtb.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

#https://free-proxy-list.net/
def get_proxy(which='free',now=False,sample=False,sampleN=0):
    which_dict={'kuai':"/home/pan/django_projects/wtb/ips_kuai.txt",
                'ihuan':"/home/pan/django_projects/wtb/ips_1000_ihuan.txt",
                'free':"/home/pan/django_projects/wtb/ips_free.txt",
                'OK':"/home/pan/django_projects/wtb/ips_OKs.txt"
               }
    #
    ippos=cache.get('ippos')
    if not ippos:
        ippos=[]
        with open(which_dict[which], 'r') as f:
            lines=f.readlines()
            for line in lines:
                line=line.strip() #remove space_\t_\n
                if len(line.split(":"))==2:
                    ippos.append(line)
        #
        cache.set('ippos', ippos, 86400)    
    #
    if sample and sampleN>0:        
        return random.sample(ippos,sampleN)
    if now:
        s=0.5+random.uniform(0, 1)
        sleep(s)
        return random.choice(ippos)
    #ippo
    old=[]
    #測試代理_________________________________________
    while True:
        fake_header = Headers(
            browser="chrome",  # Generate only Chrome UA
            os="win",  # Generate ony Windows platform
            headers=True  # generate misc headers
        )    
        UA=fake_header.generate()
        #UA = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
        #已經測過的就不要
        ippo=random.choice(ippos)
        if len(old)/len(ippos) > 0.5:
            old=[]
        if ippo in old:
            continue
        else:
            old.append(ippo)
        #    
        ip=ippo.split(":")[0]
        proxies={
                "http": "http://"+ippo,
                "https": "http://"+ippo
                }  
        #print("test:"+ippo)    
        #clear_output(wait=True)
        try:
            test=['http://icanhazip.com/','https://myip.com.tw/','https://www.showmyipaddress.eu/']
            r = requests.get(random.choice(test),#test[1], 
                             headers=UA,
                             proxies=proxies,
                             timeout=4)
            r.encoding='utf8'
            #print(r.status_code)
            #print(r.text)
            #return
            #return 22
            #
            if(r.status_code == 200 and (ip in r.text or '&#46;'.join(ip.split('.')) in r.text) ):
                print("OK:"+which+"_"+ippo)
                #print(r.status_code)
                #print(r.text)
                r.close() 
                if which!='OK':
                    fn='/home/pan/django_projects/wtb/ips_OKs.txt'
                    mode=0                    
                    if mode==0:
                        #加到最後一行
                        with open(fn, 'a+') as f:
                            f.write(ippo+"\r\n")      
                    else:
                        #刪掉第一行_加到最後一行
                        with open(fn, 'r+') as f: # open file in read / write mode
                            firstLine = f.readline() # read the first line and throw it out
                            data = f.read() # read the rest
                            data += ippo+"\r\n"
                            f.seek(0) # set the cursor to the top of the file
                            f.write(data) # write the data back
                            f.truncate() # set the file size to the current size 
                #______________
                return ippo
            else:#if not (r.status_code == 200 and ip in r.text):
                r.close()
                raise Exception('fail')
               
        except Exception as e:
            #print(str(e))
            #clear_output(wait=True) 
             
            sleep(1.5+random.uniform(0, 2))
            #sleep(0.1)
            continue
        #else:
        #    print(r.text)
        #    print("OK2:"+ip)
        #    r.close()       
        #    break    


In [2]:
if __name__ == '__main__':
    which_dict={
                #'kuai':"ips_kuai.txt",
                #'ihuan':"ips_1000_ihuan.txt",
                'free':'ips_free.txt',
                #'OK':'ips_OKs.txt'
               }
    for i in range(200):
        print(i)
        with ThreadPoolExecutor(max_workers=3) as executor:
            executor.map(get_proxy,which_dict.keys())

0
OK:free_45.7.238.250:30980
1
OK:free_117.242.37.201:57331
2
OK:free_201.204.168.106:63141
3
OK:free_120.29.155.58:31059
4
OK:free_51.255.103.170:3129
5
OK:free_82.119.170.106:8080
6
OK:free_138.255.220.5:61618
7
OK:free_103.250.158.23:61219
8
OK:free_109.199.133.161:23500
9
OK:free_190.128.26.98:56204
10
OK:free_220.135.165.38:8080
11
OK:free_103.9.188.229:36984
12
OK:free_189.57.62.146:80
13
OK:free_185.26.219.34:58250
14
OK:free_200.35.49.89:43006
15
OK:free_103.241.227.99:54073
16
OK:free_170.238.255.90:56218
17
OK:free_202.137.10.179:57338
18
OK:free_35.222.208.56:3128
19
OK:free_94.180.106.94:32767
20
OK:free_80.87.184.49:41258
21
OK:free_103.206.254.170:65103
22
OK:free_80.187.140.26:8080
23
OK:free_62.4.60.50:53281
24
OK:free_176.101.89.226:33470
25
OK:free_1.20.97.173:46790
26
OK:free_110.74.208.154:21776
27
OK:free_79.137.44.85:3129
28
OK:free_103.242.44.80:8080
29
OK:free_212.77.138.161:41258
30
OK:free_109.172.43.35:3129
31
OK:free_51.255.103.170:3129
32
OK:free_186.159.2.

In [ ]:
#kuai,ihuan,free
#%time get_proxy('OK',True,sample=True,sampleN=3)
AAA()
#print(a)

In [ ]:
with open('/home/pan/django_projects/wtb/ips_OKs.txt', 'r') as f:
    lines=f.readlines()
    for l in lines:
        print(l.strip()+"_")
        #print(l+"__")

In [ ]:
ippos=[]
with open('/home/pan/django_projects/wtb/ips_OKs.txt', 'r') as f:
    lines=f.readlines()
    for line in lines:
        line=line.strip() #remove space_\t_\n
        if len(line.split(":"))==2:
            ippos.append(line)

In [ ]:
1+2